## Getting started with Python
To get you going, take a look at [this intro video](https://youtu.be/kqtD5dpn9C8) for Python (you can skip the install step). Follow along the explanations in a [Google Colab](https://colab.research.google.com/) notebook. This will give a quick overview of Python's functionality. In case you want to look up the things you saw here later, here's a cheat sheet:

![cheat1](http://www.sixthresearcher.com/wp-content/uploads/2016/12/Python3_reference_cheat_sheet_front.png)

![cheat2](http://www.sixthresearcher.com/wp-content/uploads/2016/12/Python3_reference_cheat_sheet_back.png)

## Starting Google Earth Engine

Google Earth Engine (GEE) has a so called Application Programming Interface (API). This basically means we can send order in the form of programming language to GEE to get a specific result. In the case of GEE you can use either Javascript or Python. In the case of this course we will be using Python. Don't worry, you don't have to becomme a full fledged programmer to manage this. You just need some basics. The best way to do this is not using the native [code editor](https://code.earthengine.google.com/), but instead use [Google Colab](https://colab.research.google.com/). [Google Colab](https://youtu.be/RLYoEyIHL6A) allows you to write Python code directly in your browser and also access GEE's API from there. In this way you can easily and conveniently work with Python and GEE and you don't even have to install anything on your computer. 

In an earlier notebook we set up a GEE account. This was necessary, as GEE will always ask you to authenticate yourself when you run it. You can also trigger this explicitly:

In [ ]:
ee.Authenticate()



![auth](Assets/Notebook_2_6_1.png)

Follow the link and authenticate your account by copying the authentification code. This will allow you to access GEE from this specific notebook. In addition, it also makes sense to initialize GEE every time you run it. This loads all the functions of GEE in the working memory of Google Colab, which makes accesing them easier.



In [ ]:
ee.Initialize()

## Using Google Earth Engine interactively 
Now that we got you started on Python and GEE, next stop is getting to know the basic usage of GEE. 
We can use it either interactively or by the API. The interactive mode is enabled by the 
[*geemap* package](https://geemap.org/). *geemap* is not natively installed in Google Colab, but simply run the following code to install it:

In [ ]:
# Installs geemap package
!pip install geemap

Now let's take a look at [an introduction to *geemap*](https://youtu.be/XRFHWg_MlaQ). As you can see it is quite a powerful tool. Let's create a simple interactive map. To do this fun the following code in Google Colab. 

In [ ]:
import geemap
Map = geemap.Map(center=(40, -100), zoom=4)
Map

In addition, it also makes sense to initialize GEE every time you run it. This loads all the functions of GEE in the working memory of Google Colab, which makes accesing them easier.

Once you have done this, you should be looking at an interactive map in your notebook. This is really helpful if you want to take a quick look at data or quickly select a region where you want to get data from. 

![map](Assets/Notebook_2_6_2.png)

This allows us to work with GEE in an interactive way. However, to have more finely grained control we will be mostly using the Python API directly. Interactive mode is great if you want to test out things, while using the API allows easier reproducibility and gives you access to more computing power on the Google Servers. 

## Exercise 1: Getting feature collections 
When we use GEE we need to import the data that Google is already providing. To find the right dataset we use the [GEE Data Catalog](https://developers.google.com/earth-engine/datasets/catalog). For example, let's get the [global adminstrative units](https://developers.google.com/earth-engine/datasets/catalog/FAO_GAUL_2015_level0), so we can display the borders of states. The good thing is that this page already contains the right code snippet for  us to import this feature collection: 

In [ ]:
ee.FeatureCollection("FAO/GAUL/2015/level0")

Select [Table Schema](https://developers.google.com/earth-engine/datasets/catalog/FAO_GAUL_2015_level0#table-schema) on this site to see the different attributes of this dataset and how to access them. So, for example we can get all the country names from this by simply writing:

In [ ]:
# We get the feature collection of Administrative boundaries (level0)
countries = ee.FeatureCollection('FAO/GAUL/2015/level0').select('ADM0_NAME')

And we can also filter it to get a subset. 

In [ ]:
# We filter the featureCollection to get the feature we want
# the .eq method stands for equal. So this looks for the name Germany in the attribute ADM0_Name
germany = countries.filter(ee.Filter.eq('ADM0_NAME', 'Germany'))

And finally we can use geemap to show us the country we selected as a highlight on the map. 

In [ ]:
# Create a map
Map = geemap.Map(zoom=3)
# Add the Germany feature 
Map.addLayer(germany,{},'Countries')
# Show it
Map

Repeat this exercise with the state of the country you were born in (hint: you have to select another feature collection.

In [ ]:
# You can also use this code to select several countries at once
# Again you use the filter, but this time you select for all countries in the list you provide
countries = ee.FeatureCollection('FAO/GAUL/2015/level0').filter(ee.Filter.inList("ADM0_NAME", ['Germany', 'Brazil', 'France'))

## Exercise 2: Getting info from a feature collection
As we saw in the exercise before, feature collections contain information about their polygons like their name. We used this to select states by name. We can also ask GEE to give us all the information in a feature collection, so we can use it to select the wnats we are interest in. For this we can use the .getInfo() method. 


In [ ]:
# We get the feature collection of Administrative boundaries (level0) and ask for all values in the ADM0_Name attribute
countries = ee.FeatureCollection('FAO/GAUL/2015/level0').aggregate_array("ADM0_NAME").getInfo()

Repeat this for all the states in your home country. 

## Exercise 3: Filter based on a numerical attribute
For this we will use the dataset of the [RESOLVE ecoregions](https://developers.google.com/earth-engine/datasets/catalog/RESOLVE_ECOREGIONS_2017#table-schema). This is a global mapping of ecosystem regions. We can select it with:

In [ ]:
# Get the ecoregion data 
ecoregions = ee.FeatureCollection("RESOLVE/ECOREGIONS/2017")

Now only select the mangroves worldwide and display them:

In [ ]:
# Only select mangroves
mangroves = ecoregions.filter(ee.Filter.eq('BIOME_NAME', 'Mangroves'))
# Show them 
Map = geemap.Map()
Map.addLayer(mangroves,{},"Mangroves Worldwide")
Map

Next we only want to select all mangroves that are larger than 3 square degrees:

In [ ]:
# Only select those mangroves with a total area greater than (gt) than 3 square degrees
largest_mangroves = mangroves.filter(ee.Filter.gt("SHAPE_AREA", 3.3))
# Show them 
Map = geemap.Map()
Map.addLayer(largest_mangroves,{},"Largest Mangroves Worldwide")
Map.center_object(largest_mangroves)
Map

Repeat this exercise and find the smallest "Temperate Conifer Forests" in the world.

## Exercise 4: Grouping feature collections by attribute
Grouping is a powerful technique. It basically allows you to sort your data by a specific attribute. For example, you could group your data by country and subsequently calculate all your other attributes on a by country basis. This allows you to get a quick overview over your data. 

To do this properly we first want to exclude all shapes with missing values, as they skew our selection. 

In [ ]:
# Get the ecoregion data 
ecoregions = ee.FeatureCollection("RESOLVE/ECOREGIONS/2017")
# Remove non values for numerical attributes 
# the .And allows to select several attributes at once
# .neq stands for not equal
data_without_none = ecoregions.filter(ee.Filter.And(ee.Filter.neq("SHAPE_AREA", None),
                                                   ee.Filter.neq("SHAPE_LENGTH",None)))

Next we group by the ecoregion and calculate the mean of the shape area, to see which ecoregion is the largest one on earth. This looks complicated, but the idea is actually pretty simple. 

In [ ]:
grouped_regions = data_without_none.reduceColumns(**{
    # Define the columns we want to work with
    "selectors": ["SHAPE_AREA", "BIOME_NAME"],
    # Define what we want to calculate, e.g. min, max, mean
    # .repeat tells GEE how many of the attributes should be used for calculation (starting with the first one)
    # .group defines that we want to group the results by something
    "reducer": ee.Reducer.mean().repeat(1).group(**{
        # the group field refers to the line with "selectors" and starts counting at 0
        # the field we select will be used for grouping, so "BIOME_NAME" in this case
        "groupField":1,
        # groupName is simply a lable we want to use for the groups
        "groupName":"Ecoregion"
    })
})
grouped_regions.getInfo()

Repeat this exercise by selecting a new feature collection from the [data calalog](https://developers.google.com/earth-engine/datasets/catalog) and collect the max value for a meaningful grouped attribute.

## Creating your own GeoJSON
Basically a simple syntax to store geographical data. For more information, take a look at [this video](https://youtu.be/8RPfrhzRw2s). When using GEE or other GIS software it can often be helpful to create your own GeoJSONs quickly. You can use this to quickly create a polygon for a area you are interested in. To do this we can use [geojson.io](http://geojson.io/#map=2/20.0/0.0) ([explanation video](https://youtu.be/9ZkXYnl3-yg)). If you want to use it in Google Colab, we need to copy the output of the website. 

In [ ]:
my_geojson = {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "type": "Polygon",
        "coordinates": [
          [
            [
              -12.83203125,
              13.325484885597936
            ],
            [
              16.5234375,
              13.325484885597936
            ],
            [
              16.5234375,
              33.211116472416855
            ],
            [
              -12.83203125,
              33.211116472416855
            ],
            [
              -12.83203125,
              13.325484885597936
            ]
          ]
        ]
      }
    }
  ]
}

To convert it to a usable geometry, we can just use geemap. 

In [ ]:
area_of_interest = geemap.geojson_to_ee(my_geojson)
print(area_of_interest)

## Clipping a raster by a vector in Google Earth Engine
Clipping rasters to the extents you need is one of the most common operations in GIS, which means you can also do it in GEE. For this we will use the GeoJSON file we just read in and a [DEM](https://developers.google.com/earth-engine/datasets/catalog/CGIAR_SRTM90_V4). 

In [ ]:
dem = ee.Image("CGIAR/SRTM90_V4") 
dem_clipped = dem.clip(area_of_interest)
Map = geemap.Map()
Map.addLayer(dem_clipped,{},"Area of Interest")
Map.center_object(dem_clipped)
Map

Here's the code we need to sample from a specific point from a single image. 

In [ ]:
scale = 1000  # scale in meters
# Calculate and print the mean value of the LST collection at the point.
lst_urban_point = lst.mean().sample(u_poi, scale).first().get('LST_Day_1km').getInfo()
print('Average daytime LST at urban point:', round(lst_urban_point*0.02 -273.15, 2), '°C')

Let's go through this in detail:
* lst.mean() calculates the mean values for an image collection and returns a single image that only contains the average values. In this case it is the average temperature for every point for the whole time period. 
* .sample() can samples a point in an image. For this it needs a positions (u_poi) and a scale. GEE saves images in different resolutions. The scale keywords tells GEE the resolution we would like to get the image in and it will return the resolution that is closest to the scale we asked for. It always returns a feature collection, even if we just ask for one element as in this case.
* .first() returns the first element of a feature collection, which is exactly what we want here. 
* .get() selects the band we would like to look at
* .getInfo() retrieves the info from the server, so we can work with it in the notebook

## Common operations on geometries

Geometries (things like lines, points and polygons) can be queried for their properties and perform simple spatial analysises like intersections. Here is a collection of of common operations:


## Filtering

## Operations on image collections

## Exporting data from Google Earth Engine

## Using Google Earth Engine in QGIS